# Introduction

This notebook serves as an example 

In [1]:
library(splines)
library(MASS)
library(LFBayes)
library(pracma)

source("Simulation_funcs.R")
errorvar <- .025
SS <- 20
TT <- 20
t <- seq(from = 0, to = 1, length.out = TT)
s <- seq(from = 0, to = 1, length.out = SS)
n <- 60
tt <- list()
tt[[1]] <- 1:(TT*SS)
tt <- rep(tt, n)
p1 <- 12
p2 <- 12
q1 <- 3
q2 <- 3
Bt <- bs(t, df = p1, intercept = TRUE)
Bs <- bs(s, df = p2, intercept = TRUE)

Warning message:
"package 'LFBayes' was built under R version 3.6.2"
Warning message:
"package 'pracma' was built under R version 3.6.2"


In [2]:
H <- GenerateH(q1, q2)
mu1 <- GenerateMu1(s,t)
Lambda <- Loading.Matern(t, p1, q1, Bt)
Gamma <- Loading.Brown.Bridge(s, p2, q2)
Cov <- kronecker(Bs%*%Gamma, Bt%*%Lambda)%*%H%*%t(kronecker(Bs%*%Gamma, Bt%*%Lambda)) + errorvar * diag(SS * TT)

In [3]:
x <- mvrnorm(n, mu  = as.vector(mu1), Sigma = Cov)
sx <- sd(x)
mx <- mean(x)
x <- (x-mx)/sx
Smooth_scaled_cov <- (Cov - errorvar * diag(SS * TT)) / sx^2
mu <- (mu1 - mx)/sx
y <- lapply(1:n, function(i) x[i,])
missing <- list()
for(ii in 1:n){
  missing[[ii]] <- numeric(0)
}
X <- cbind(rep(1, n))

In [4]:
q1_fit <- 6
q2_fit <- 6
p1_fit <- 14
p2_fit <- 14
Bs_fit <- bs(s, df = p2_fit, intercept = TRUE)
Bt_fit <- bs(t, df = p1_fit, intercept = TRUE)
iter <- 500 # Number of iterations
burnin <- 200 # Burnin iterations
thin <- 1 # Thinning for each chain
nchain <- 1 # Number of chains
neig <- 3 # Number of eigenfunctions for inference
LF_fit <- mcmcWeakChains(y, missing, X, Bs_fit, Bt_fit, q1_fit, q2_fit, iter, thin, burnin, nchain)
Post_proc <- eigenLFChains(Bs_fit, Bt_fit, LF_fit, neig, iter, burnin, nchain,s,t)

Starting MCMC...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
All done!
Starting post-processing...
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
All done!
